In [1]:
import pandas as pd
import numpy as np
import pathlib
from time import time
import warnings
import shutil
from xlrd import XLRDError
warnings.filterwarnings("ignore")

In [2]:
ptr_df_columns = ['TEST_NUM', 'SITE_NUM', 'TEST_FLG', 'RESULT','TEST_TXT',
                  'LO_LIMIT', 'HI_LIMIT', 'UNITS']
ftr_df_columns = ['TEST_NUM', 'SITE_NUM', 'TEST_FLG','TEST_TXT']
prr_df_columns = ['PART_ID', 'PART_FLG', 'NUM_TEST']
mir_df_columns = ['LOT_ID','PART_TYP','JOB_NAM','OPER_NAM','TSTR_TYP','EXEC_VER','TST_TEMP','PKG_TYP','RTST_COD']
config_df_columns = ['TEST_NUM','TEST_TXT',"T_Type","LO_LIMIT",'HI_LIMIT',"MAR_HI_UP_LIMIT","MAR_HI_DOWN_LIMIT","MAR_LO_UP_LIMIT","MAR_LO_DOWN_LIMIT",'UNITS']
final_df_columns = ['TEST_NUM', 'TEST_TXT','PART_ID', 'Status',"LO_LIMIT",'HI_LIMIT',"MAR_HI_UP_LIMIT","MAR_HI_DOWN_LIMIT","MAR_LO_UP_LIMIT","MAR_LO_DOWN_LIMIT", 'UNITS', 'P_Type']
non_exe_columns = ["TEST_NUM",'TEST_TXT', "SITE_NUM","TEST_FLG","RESULT","LO_LIMIT","HI_LIMIT","MAR_HI_UP_LIMIT","MAR_HI_DOWN_LIMIT","MAR_LO_UP_LIMIT","MAR_LO_DOWN_LIMIT","UNITS","P_Type","PART_ID","STATUS"]

In [3]:
def get_FTR_df(masterfile):
    flag = 0
    try:
        df = pd.read_excel(masterfile, sheet_name = 'FTR')[ftr_df_columns]
    except XLRDError:
        print ("FTR Sheet not exists")
        df = pd.DataFrame(columns = ftr_df_columns)
        flag = 1
    return df,flag

In [4]:
def getDataset(masterfile,config_file):
    # Reading all files
    ptr_df = pd.read_excel(masterfile, sheet_name = 'PTR')[ptr_df_columns]

    #ftr_df = pd.read_excel(masterfile, sheet_name = 'FTR')[ftr_df_columns]
    ftr_df,flag = get_FTR_df(masterfile)
    prr_df = pd.read_excel(masterfile, sheet_name = 'PRR')[prr_df_columns]
    mir_df = pd.read_excel(masterfile, sheet_name = 'MIR')[mir_df_columns]    
    config_df = pd.read_csv(config_file)
    #print('config_df :',config_df)
    # empty data frame template
    df_final = pd.DataFrame(columns=final_df_columns)
    non_exe_df = pd.DataFrame(columns=non_exe_columns)
    return ptr_df, ftr_df, prr_df,mir_df,config_df, df_final, non_exe_df,flag

In [ ]:
def getExpandReport(df_final, prr_df, config_df):
    # Expand result and will bring PART_ID in final data 
    for row in prr_df.itertuples():
        # PART_FLG == 0 denotes those record is pass in PRR data
        if row.PART_FLG == 0:         
            part_1 = config_df[['TEST_NUM']]
            part_1['PART_ID'] = row.PART_ID
            part_1['Status'] = 'Pass'
            part_1['TEST_TXT'] = config_df['TEST_TXT']
            part_1['LO_LIMIT'] = config_df['LO_LIMIT']
            part_1['HI_LIMIT'] = config_df['HI_LIMIT']
            # part_1['MAR_LO_LIMIT'] = config_df['MAR_LO_LIMIT']
            # part_1['MAR_HI_LIMIT'] = config_df['MAR_HI_LIMIT']
            part_1['MAR_HI_UP_LIMIT'] = config_df['MAR_HI_UP_LIMIT']
            part_1['MAR_HI_DOWN_LIMIT'] = config_df['MAR_HI_DOWN_LIMIT']
            part_1['MAR_LO_UP_LIMIT'] = config_df['MAR_LO_UP_LIMIT']
            part_1['MAR_LO_DOWN_LIMIT'] = config_df['MAR_LO_DOWN_LIMIT']
            part_1['UNITS'] = config_df['UNITS']
            df_final = df_final.append(part_1)
        else:        
            part_1 = config_df[['TEST_NUM']]
            part_1['PART_ID'] = row.PART_ID
            part_1['Status'] = 'Fail'
            part_1['TEST_TXT'] = config_df['TEST_TXT']
            part_1['LO_LIMIT'] = config_df['LO_LIMIT']
            part_1['HI_LIMIT'] = config_df['HI_LIMIT']
            # part_1['MAR_LO_LIMIT'] = config_df['MAR_LO_LIMIT']
            # part_1['MAR_HI_LIMIT'] = config_df['MAR_HI_LIMIT']
            part_1['MAR_HI_UP_LIMIT'] = config_df['MAR_HI_UP_LIMIT']
            part_1['MAR_HI_DOWN_LIMIT'] = config_df['MAR_HI_DOWN_LIMIT']
            part_1['MAR_LO_UP_LIMIT'] = config_df['MAR_LO_UP_LIMIT']
            part_1['MAR_LO_DOWN_LIMIT'] = config_df['MAR_LO_DOWN_LIMIT']
            part_1['UNITS'] = config_df['UNITS']
            #print("before : \n",part_1.iloc[0:row.NUM_TEST,[2]])
            #print("Part_ID : ",row.PART_ID)
            part_1.iloc[0:row.NUM_TEST,[2]] = 'Pass'      # causing issue
            #print("part_1.iloc[0:row.NUM_TEST,[2]] : \n",part_1.iloc[0:row.NUM_TEST,[2]])
            
            df_final = df_final.append(part_1)
    return df_final